In [ ]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
%matplotlib inline

def soup_of_steam(game_type,page):
    page = int(page)-1
    url = f'https://store.steampowered.com/tags/en/{game_type}/#p={page}&tab=TopSellers'
    print(url)
    request_result = requests.get(url)
    soup= BeautifulSoup(request_result.content, 'lxml')
    return soup

soup = soup_of_steam('Simulation','3')

game_name_list = []
for listing in soup.find_all('div', {'id':'TopSellersRows'}):
    for gamenames in listing.find_all('div', {'class':'tab_item_name'}):
        game_names = gamenames.text.strip()
        game_name_list.append(game_names)

game_price_list = []
for listing in soup.find_all('div', {'id':'TopSellersRows'}):
    for gameprices in listing.find_all('div', {'class':'discount_final_price'}):
        game_prices = gameprices.text.strip()
        if game_prices == 'Free to Play':
            game_prices = 'CDN$ 0.00'
        else:
            game_prices = game_prices
        game_price_list.append(game_prices)

game_prices_number = []
for i in range(0,len(game_price_list)):
    game_price_number = float(game_price_list[i][5:])
    game_prices_number.append(game_price_number)
    
def merge(list1,list2,list3):
    merged_list = [(list1[i],list2[i],list3[i]) for i in range(0,len(list1))]
    return merged_list

game_name_price = merge(game_name_list,game_price_list,game_prices_number)

sortgames = sorted(game_name_price,key=lambda x:x[2])

import pandas as pd

df = pd.DataFrame(sortgames, columns=['game name', 'price','price number'])

df = df.drop('price number',axis = 1)

df